In [2]:
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 6.1MB/s 


In [3]:
(trainx, trainy), (testx, testy) = keras.datasets.fashion_mnist.load_data()
trainx = trainx / 255.0
testx = testx/255.0

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
def build(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  #continous values controlled by steps
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))
  # continous or discrete parameters
  # discrete-> choice
  hp_lr = hp.Choice('lr', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=hp_lr),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model


In [8]:
tuner = kt.Hyperband(build,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hpdir',
                     project_name='mnisttuning')

In [9]:
# Early Stopping
# relieve algos that are wasting resources
# patience=> how many iters to ignore before checking for performance for early stopping
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
tuner.search(trainx, trainy, epochs=50, validation_split=0.2, callbacks=[earlystop])

Trial 24 Complete [00h 00m 10s]
val_accuracy: 0.846750020980835

Best val_accuracy So Far: 0.8852499723434448
Total elapsed time: 00h 05m 41s

Search: Running Trial #25

Hyperparameter    |Value             |Best Value So Far 
units             |256               |160               
lr                |0.01              |0.001             
tuner/epochs      |4                 |10                
tuner/initial_e...|0                 |4                 
tuner/bracket     |1                 |2                 
tuner/round       |0                 |2                 

Epoch 1/4
1493/1500 [============================>.] - ETA: 0s - loss: 1.0668 - accuracy: 0.7269